In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.auto import tqdm
import sys
import matplotlib
import seaborn as sns
import pandas as pd
import numpy as np
import datetime as dt
import os
from multiprocessing import Pool
from tqdm.auto import tqdm

pd.set_option('display.max_columns', 50)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


# Import Data

In [2]:
## Read in all files from TAQ

# Fix forward/backslashes in directory name
taq_folder = '../data/taq/'
if sys.platform != 'linux':
    taq_folder = taq_folder.replace('/', '\\')
print(f'Reading data from: {taq_folder}')

# Read files
taq_files = [x[2] for x in tqdm(os.walk(taq_folder))][0]
ticker_df = pd.concat([pd.read_csv(taq_folder + taq_file) for taq_file in taq_files]).astype({'DATE':np.int32,'PRICE':np.float32})

Reading data from: ..\data\taq\


In [26]:
# Read whatever csv from WRDS here
ticker_df = pd.read_csv('../data/3dce8551ace67a48_csv.zip', nrows = 10000, low_memory = False)
ticker_df.head()

,DATE,TIME_M,EX,SYM_ROOT,SYM_SUFFIX,PRICE
0,20190920,9:30:00.025559149,Q,GOOG,NaN,1234.75
1,20190920,9:30:00.271118326,Q,GOOG,NaN,1234.01
2,20190920,9:30:00.271126947,K,GOOG,NaN,1233.12
3,20190920,9:30:00.271166170,Q,GOOG,NaN,1234.01
4,20190920,9:30:00.271169340,Q,GOOG,NaN,1234.00


In [27]:
# Concatenated file info
print('Tickers: ' + ', '.join(ticker_df['SYM_ROOT'].unique()))
#print('Dates: ' + ', '.join(np.sort(ticker_df['DATE'].astype(str).unique())))

ticker_df.head()

Tickers: GOOG


,DATE,TIME_M,EX,SYM_ROOT,SYM_SUFFIX,PRICE
0,20190920,9:30:00.025559149,Q,GOOG,NaN,1234.75
1,20190920,9:30:00.271118326,Q,GOOG,NaN,1234.01
2,20190920,9:30:00.271126947,K,GOOG,NaN,1233.12
3,20190920,9:30:00.271166170,Q,GOOG,NaN,1234.01
4,20190920,9:30:00.271169340,Q,GOOG,NaN,1234.00


In [28]:
ticker_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DATE        10000 non-null  int64  
 1   TIME_M      10000 non-null  object 
 2   EX          10000 non-null  object 
 3   SYM_ROOT    10000 non-null  object 
 4   SYM_SUFFIX  0 non-null      float64
 5   PRICE       10000 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 468.9+ KB


# Clean Data

## Fix/Add columns

In [29]:
# Sort columns
ticker_df = ticker_df.sort_values(by = ['DATE'])
# Rename columns
ticker_df.rename(columns = {'SYM_ROOT': 'ticker', 'PRICE': 'price','SIZE':'size'}, inplace = True)

In [30]:
## Ticker Descriptions
# Long ticker descriptions
ticker_desc_dict = {
'FB':'Facebook',
'AMZN' :'Amazon',
'NFLX':'Netflix',
'AAPL':'Apple',
'GOOG':'Google'
}

ticker_df["ticker_desc"] = ticker_df["ticker"].apply(lambda x: ticker_desc_dict.get(x, "NA"))

In [31]:
# Convert date and time info to pandas datetime
ticker_df["datetime"] = pd.to_datetime(ticker_df['DATE'].astype(str) + ' ' + ticker_df['TIME_M'])
ticker_df['date'] = (ticker_df['datetime'].dt.date).astype(str)

# Ckeck Exchange Data

In [32]:
ordered_tickers=ticker_df.query("date=='2020-09-16'")['ticker'].unique()
for ticker in ordered_tickers:
    temp_df=ticker_df.query("date == '2020-09-16' & ticker == @ticker").sort_values(by='datetime').copy()
    g = sns.FacetGrid(temp_df, col = 'EX', col_wrap = 5)
    g = g.map(plt.plot, 'datetime', 'price')

In [33]:
# Drop weird exchange
ticker_df = ticker_df.query('EX != ["D","A","M","L"]').copy()

In [34]:
ticker_df['EX'].unique()

array(['Q', 'Z', 'K', 'V', 'B', 'J', 'Y', 'P', 'N', 'X', 'C'],
      dtype=object)

In [35]:
ticker_df.head()

,DATE,TIME_M,EX,ticker,SYM_SUFFIX,price,ticker_desc,datetime,date
0,20190920,9:30:00.025559149,Q,GOOG,NaN,1234.75,Google,2019-09-20 09:30:00.025559149,2019-09-20
6663,20190920,10:38:51.001043789,Q,GOOG,NaN,1236.02,Google,2019-09-20 10:38:51.001043789,2019-09-20
6664,20190920,10:38:51.001074368,Q,GOOG,NaN,1236.02,Google,2019-09-20 10:38:51.001074368,2019-09-20
6665,20190920,10:38:51.001077914,Q,GOOG,NaN,1236.02,Google,2019-09-20 10:38:51.001077914,2019-09-20
6666,20190920,10:38:51.001116621,Q,GOOG,NaN,1236.02,Google,2019-09-20 10:38:51.001116621,2019-09-20


# Resample prices

In [36]:
def resample_data(data,sampling_freq):
    # Fix datetime type
    data['datetime']=pd.to_datetime(data['datetime'])
    data=data.sort_values(by=["ticker", "date", "datetime"])
    
    # Resample on a given basis
    resample=(
    data.set_index("datetime")
    .groupby(["ticker", "ticker_desc", "date"])
    .resample(sampling_freq,label = 'right')["price"]
    .last()
    .reset_index().copy()
    )
    
    # Forward fill missing prices for particular intervals using last observed price
    resample["price"] = (
    resample.sort_values(by="datetime")
    .groupby(["ticker", "ticker_desc", "date"])["price"]
    .fillna(method="ffill")
    )
    
    
    # Add price of time == 0
   
    df = data.groupby(["ticker", "ticker_desc", "date"]).first().reset_index().copy()
    df=(
    df.set_index("datetime")
    .groupby(["ticker", "ticker_desc", "date"])
    .resample(sampling_freq,label = 'left')["price"]
    .last()
    .reset_index().copy()
    )
    #df = df[["ticker", "ticker_desc", "date", "datetime", "price"]]
    resample = pd.concat([df,resample]).copy()
    
    
    # Add returns
    resample = resample.sort_values(by=["ticker", "date", "datetime"])
    resample["log_price"] = np.log(resample["price"])
    resample["return"] = resample.groupby(["ticker", "date"])["log_price"].diff(1).fillna(0)
    
    return resample

In [13]:
ticker_df1 = ticker_df.sort_values(by=['date','datetime']).copy()

In [14]:
ticker_df1.to_csv('../data/clean/ticker1.csv', index = False)

In [14]:
for i in tqdm([1,5]):
    df = resample_data(ticker_df,sampling_freq=str(i)+'s')
    df.to_csv('../data/clean/ticker1_{}s.csv'.format(str(i)), index = False)

In [37]:
resample_data(ticker_df, '1s').head(n=20)

,ticker,ticker_desc,date,datetime,price,log_price,return
0,GOOG,Google,2019-09-20,2019-09-20 09:30:00,1234.750,7.118624,0.000000
0,GOOG,Google,2019-09-20,2019-09-20 09:30:01,1235.350,7.119110,0.000486
1,GOOG,Google,2019-09-20,2019-09-20 09:30:02,1236.160,7.119765,0.000655
2,GOOG,Google,2019-09-20,2019-09-20 09:30:03,1235.180,7.118972,-0.000793
3,GOOG,Google,2019-09-20,2019-09-20 09:30:04,1235.530,7.119255,0.000283
4,GOOG,Google,2019-09-20,2019-09-20 09:30:05,1235.540,7.119263,0.000008
5,GOOG,Google,2019-09-20,2019-09-20 09:30:06,1235.530,7.119255,-0.000008
6,GOOG,Google,2019-09-20,2019-09-20 09:30:07,1235.530,7.119255,0.000000
7,GOOG,Google,2019-09-20,2019-09-20 09:30:08,1235.530,7.119255,0.000000
8,GOOG,Google,2019-09-20,2019-09-20 09:30:09,1235.540,7.119263,0.000008


In [16]:
df =   pd.read_csv('../data/clean/ticker_1s/ticker1_1s.csv')
df.head()

,ticker,ticker_desc,date,datetime,price,log_price,return
0,GOOG,Google,2019-09-20,2019-09-20 09:30:00,1234.26,7.118227,0.000000
1,GOOG,Google,2019-09-20,2019-09-20 09:30:01,1235.35,7.119110,0.000883
2,GOOG,Google,2019-09-20,2019-09-20 09:30:02,1236.05,7.119676,0.000566
3,GOOG,Google,2019-09-20,2019-09-20 09:30:03,1235.18,7.118972,-0.000704
4,GOOG,Google,2019-09-20,2019-09-20 09:30:04,1236.01,7.119644,0.000672


In [17]:
ticker_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20652702 entries, 0 to 28254219
Data columns (total 9 columns):
DATE           int32
TIME_M         object
EX             object
ticker         object
SYM_SUFFIX     object
price          float32
ticker_desc    object
datetime       datetime64[ns]
date           object
dtypes: datetime64[ns](1), float32(1), int32(1), object(6)
memory usage: 1.4+ GB


In [18]:
df.head()

,ticker,ticker_desc,date,datetime,price,log_price,return
0,GOOG,Google,2019-09-20,2019-09-20 09:30:00,1234.26,7.118227,0.000000
1,GOOG,Google,2019-09-20,2019-09-20 09:30:01,1235.35,7.119110,0.000883
2,GOOG,Google,2019-09-20,2019-09-20 09:30:02,1236.05,7.119676,0.000566
3,GOOG,Google,2019-09-20,2019-09-20 09:30:03,1235.18,7.118972,-0.000704
4,GOOG,Google,2019-09-20,2019-09-20 09:30:04,1236.01,7.119644,0.000672
